# Objective
- OCR capabilities demonstration
- Comparison between Tesseract and Donut (if applicable)

In [5]:
!python -m venv venv
!source venv/bin/activate

In [6]:
!pip install --upgrade datafog --quiet


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [7]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install nest_asyncio

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
from datafog import DataFog
client = DataFog(operations="extract_text")

In [14]:
import re

scanned_medical_record = "https://s3.amazonaws.com/thumbnails.venngage.com/template/dc377004-1c2d-49f2-8ddf-d63f11c8d9c2.png"

def is_valid_url(url):
    regex = re.compile(
        r'^(?:http|ftp)s?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'  # domain...
        r'localhost|'  # localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}|'  # ...or ipv4
        r'\[?[A-F0-9]*:[A-F0-9:]+\]?)'  # ...or ipv6
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, url) is not None

async def run_ocr_pipeline_demo():
    if not is_valid_url(scanned_medical_record):
        print("Invalid URL:", scanned_medical_record)
        return

    results = await client.run_ocr_pipeline(image_urls=[scanned_medical_record])
    print("OCR Pipeline Results:", results)

loop = asyncio.get_event_loop()
loop.run_until_complete(run_ocr_pipeline_demo())

OCR Pipeline Results: ["MEDICAL BILLING INVOICE\n\nPATIENT INFORMATION\n\nKemba Harris\n(855) 595-5999\n\n11 Rosewood Drive,\nCollingwood, NY 33580\n\nPERSCRIBING PHYSICIAN'S INFORMATION\n\nDr. Alanah Gomez\n(855) 505-5000\n\n102 Trope Street,\nNew York, NY 45568\n\nDATE\n\n07/01/23\n\nINVOICE NUMBER\n\n12245\n\nINVOICE DUE DATE\n\n07/30/23\n\nAmount DUE\n\n$1,745.00\n\nITEM DESCRIPTION AMOUNT\nFull Check Up Full body check up $745.00\n1,000.00\nEar & Throat Examination Infection check due to inflammation sy\nNOTES SUBTOTAL $745.00\nA prescription has been written out for patient, TAXRATE 9%\nfor an acute throat infection.\nTAX $157.05\nTOTAL $1,902.05\n\nConcordia Hill Hospital\n\nwww.concordiahill.com\n\nFor more information or any issues or concerns,\nemail us at invoices@concordiahill.com\n\n"]
